In [4]:
# 将原始图片资源里的人脸抽取做训练素材

In [5]:
#coding: UTF-8

# 包含库 ######################################################
# 绘制图片需要
from matplotlib import pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm

import os
import sys
sys.path.append('/usr/local/lib64')
sys.path.append('/home/wilson/faceEmotionCppLinux/lib')

# 引入功能库
import extractFace as feExtracter

In [6]:
# 获取文件名(带后缀)
def getFileName(fileUrl):
    filepath, tmpfilename = os.path.split(fileUrl)
    return tmpfilename
# print(getFileName("/home/wilson/aaa.png"))

# 将脸部存储到指定目录
def saveFace(faceInfo, faceImageSavePath):
    # 原图信息
    imagePath = bytes.decode(faceInfo.imagePath)
    image = cv2.imread(imagePath)
    sp = image.shape            #获取图像形状：返回【行数值，列数值】列表
    height = sp[0]                 #图像的高度（行 范围）
    width = sp[1]                 #图像的宽度（列 范围）

    # 修正截取位置
    top = max(0, faceInfo.top)
    bottom = min(height, faceInfo.bottom)
    left = max(0, faceInfo.left)
    right = min(width, faceInfo.right)
    cropImg = image[top:bottom, left:right] #裁剪图像
    #plt.imshow(cropImg)
    #plt.show()
    
    # 检验解码目录
    if not os.path.exists(faceImageSavePath):                                                                                                                    
         os.makedirs(faceImageSavePath)
    
    imageSavePath = faceImageSavePath+getFileName(imagePath)
    print(imageSavePath)
    cv2.imwrite(imageSavePath,cropImg)  #写入图像路径
    
# 抽取人脸
def extractFace(instance, nodeName, dirPath, faceImageSavePath):
    response = feExtracter.extractFace(instance, dirPath)
    if response.count < 1:
        print("no any faces extracted")
    else:
        print(nodeName, response.count, "face extracted")
        for k in tqdm(range(response.count)):
            e = response.elems[k]
            saveFace(e, faceImageSavePath + nodeName + "/")
        feExtracter.releaseFaceExtractResponse(instance, response)

In [7]:
# 配置原始资源目录，和人脸存放目录
rawImagePath = '/home/wilson/TFResFaceEmotion/raw/'
faceImageSavePath = '/home/wilson/TFResFaceEmotion/extract/'

# 创建实例
instance = feExtracter.createInstance()

# 遍历原始图片的根目录
for nodeName in os.listdir(rawImagePath):
    nodePath = rawImagePath + nodeName
    if os.path.isdir(nodePath): # 这是表情目录
        extractFace(instance, nodeName, nodePath, faceImageSavePath)

feExtracter.releaseInstance(instance)
print("Done")

OSError: libfaceEmotion.so: cannot open shared object file: No such file or directory